<a href="https://colab.research.google.com/github/KaggleStudy4th/4th_kaggle_study/blob/main/%EC%8B%A4%EC%8A%B5%EC%BD%94%EB%93%9C/20%EC%A3%BC%EC%B0%A8/DNN%EC%9D%84_%EC%9D%B4%EC%9A%A9%ED%95%9C_MNIST_%EC%86%90%EA%B8%80%EC%94%A8_%EB%B6%84%EB%A5%98.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
plt.style.use('ggplot')
import warnings
warnings.filterwarnings('ignore')

In [2]:
os.environ['KAGGLE_USERNAME']='jiinpp'
os.environ['KAGGLE_KEY']="b06d525a9f66198051ccb648b091778a"

In [3]:
!kaggle competitions download -c 2021-ai-w7-p1

 51% 9.00M/17.8M [00:00<00:00, 61.4MB/s]
100% 17.8M/17.8M [00:00<00:00, 102MB/s] 


In [4]:
!unzip '*.zip'

Archive:  2021-ai-w7-p1.zip
  inflating: mnist_test.csv          
  inflating: mnist_train.csv         
  inflating: submission.csv          


In [5]:
train=pd.read_csv('mnist_train.csv')
test=pd.read_csv('mnist_test.csv')
sample=pd.read_csv('submission.csv')

In [6]:
print(train.shape)
print(test.shape)

(60000, 786)
(10000, 785)


In [9]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60000 entries, 0 to 59999
Columns: 786 entries, Unnamed: 0 to 784
dtypes: float64(785), int64(1)
memory usage: 359.8 MB


In [10]:
train.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,775,776,777,778,779,780,781,782,783,784
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
3,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,9.0


In [11]:
test.head()

,Unnamed: 0,0,1,2,3,4,5,6,7,8,...,774,775,776,777,778,779,780,781,782,783
0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [7]:
train.drop('Unnamed: 0',axis=1,inplace=True)
test.drop('Unnamed: 0',axis=1,inplace=True)

In [8]:
X_train=train.drop('784',axis=1)
y_train=train['784']
X_test=test

In [10]:
X_train=np.array(X_train)
X_test=np.array(X_test)

In [9]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import random

In [11]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

random.seed(777)
torch.manual_seed(777)
if device == 'cuda':
  torch.cuda.manual_seed_all(777)

In [12]:
X_train=torch.tensor(X_train,dtype=torch.float)/255
X_test=torch.tensor(X_test,dtype=torch.float)/255
y_train=torch.tensor([int(x) for x in y_train])

In [13]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)

torch.Size([60000, 784])
torch.Size([60000])
torch.Size([10000, 784])


In [14]:
learning_rate=0.001
epochs=15
batch_size=100
drop_prob=0.3

In [15]:
from torch.utils.data import DataLoader,TensorDataset

train_dataset=TensorDataset(X_train,y_train)
train_loader=DataLoader(train_dataset,batch_size=batch_size)

In [16]:
linear1=torch.nn.Linear(784,512,bias=True)
linear2=torch.nn.Linear(512,512,bias=True)
linear3=torch.nn.Linear(512,512,bias=True)
linear4=torch.nn.Linear(512,512,bias=True)
linear5=torch.nn.Linear(512,512,bias=True)
linear6=torch.nn.Linear(512,10,bias=True)
relu=torch.nn.ReLU()
dropout=torch.nn.Dropout(p=drop_prob)

In [17]:
torch.nn.init.xavier_uniform_(linear1.weight)
torch.nn.init.xavier_uniform_(linear2.weight)
torch.nn.init.xavier_uniform_(linear3.weight)
torch.nn.init.xavier_uniform_(linear4.weight)
torch.nn.init.xavier_uniform_(linear5.weight)
torch.nn.init.xavier_uniform_(linear6.weight)


Parameter containing:
tensor([[-0.0868, -0.0647,  0.0561,  ..., -0.0482, -0.0441,  0.0159],
        [-0.0514,  0.0601, -0.0592,  ...,  0.0597,  0.0598, -0.0193],
        [-0.0517,  0.0386,  0.0422,  ...,  0.0853, -0.0975, -0.0138],
        ...,
        [ 0.0367,  0.0907,  0.0141,  ...,  0.0883, -0.0469, -0.0800],
        [-0.0794,  0.0379, -0.0432,  ..., -0.0119,  0.0037, -0.0291],
        [-0.0354, -0.0525,  0.0778,  ..., -0.0858, -0.0016, -0.0738]],
       requires_grad=True)

In [18]:
model=torch.nn.Sequential(linear1,relu,dropout,
                          linear2,relu,dropout,
                          linear3,relu,dropout,
                          linear4,relu,dropout,
                          linear5,relu,dropout,
                          linear6).to(device)

In [19]:
model

Sequential(
  (0): Linear(in_features=784, out_features=512, bias=True)
  (1): ReLU()
  (2): Dropout(p=0.3, inplace=False)
  (3): Linear(in_features=512, out_features=512, bias=True)
  (4): ReLU()
  (5): Dropout(p=0.3, inplace=False)
  (6): Linear(in_features=512, out_features=512, bias=True)
  (7): ReLU()
  (8): Dropout(p=0.3, inplace=False)
  (9): Linear(in_features=512, out_features=512, bias=True)
  (10): ReLU()
  (11): Dropout(p=0.3, inplace=False)
  (12): Linear(in_features=512, out_features=512, bias=True)
  (13): ReLU()
  (14): Dropout(p=0.3, inplace=False)
  (15): Linear(in_features=512, out_features=10, bias=True)
)

In [20]:
loss=torch.nn.CrossEntropyLoss().to(device)
optimizer=torch.optim.Adam(model.parameters(),lr=learning_rate)

In [22]:
total_batch=len(train_loader)

model.train()
for epoch in range(epochs):
  avg_cost=0
  for X,Y in train_loader:

    X=X.to(device)
    Y=Y.to(device)

    optimizer.zero_grad()
    hypothesis=model(X)
    cost=loss(hypothesis,Y)
    cost.backward()
    optimizer.step()

    avg_cost+=cost/total_batch

  print('epoch:','%04d' %(epoch+1),'cost=','{:.9f}'.format(avg_cost))



epoch: 0001 cost= 0.050752636
epoch: 0002 cost= 0.048208930
epoch: 0003 cost= 0.047745049
epoch: 0004 cost= 0.049601488
epoch: 0005 cost= 0.047697090
epoch: 0006 cost= 0.044643741
epoch: 0007 cost= 0.045692336
epoch: 0008 cost= 0.046869706
epoch: 0009 cost= 0.041823957
epoch: 0010 cost= 0.038813420
epoch: 0011 cost= 0.044561718
epoch: 0012 cost= 0.039187055
epoch: 0013 cost= 0.038225032
epoch: 0014 cost= 0.043835480
epoch: 0015 cost= 0.034172468


In [23]:
with torch.no_grad():
  model.eval()

  X_test=X_test.to(device)
  y=model(X_test)
  y=torch.argmax(y,1)

In [24]:
y=y.cpu().numpy()

In [25]:
sample['Label']=y

In [26]:
sample

,id,Label
0,0,7
1,1,2
2,2,1
3,3,0
4,4,4
...,...,...
9995,9995,2
9996,9996,3
9997,9997,4
9998,9998,5


In [27]:
sample.to_csv('sample1.csv',index=False)